**************************************************INITIALIZATION*********************************************************

In [ ]:
from tjmonopix.tjmonopix import TJMonoPix
import numpy as np
#from tjmonopix.analysis.plotting import plotting
import time

#plot=plotting()
chip = TJMonoPix(conf="../tjmonopix/tjmonopix_mio3.yaml",no_power_reset=False)
chip.init(fl="EN_HV")

# chip['data_rx'].CONF_START_FREEZE = 15 #default 3
# chip['data_rx'].CONF_STOP_FREEZE = 100 #default 40
# #chip['data_rx'].CONF_STOP_FREEZE = 250 #default 40
# chip['data_rx'].CONF_START_READ = 35 #default 6
# chip['data_rx'].CONF_STOP_READ = 37 #default 7
# chip['data_rx'].CONF_STOP = 105 #default 45
# #chip['data_rx'].CONF_STOP = 255 #default 45

chip['data_rx'].CONF_START_FREEZE = 64 #default 3
chip['data_rx'].CONF_STOP_FREEZE = 100 #default 40
chip['data_rx'].CONF_START_READ = 66 #default 6
chip['data_rx'].CONF_STOP_READ = 68 #default 7
chip['data_rx'].CONF_STOP = 105 #default 45


#-------------------------------------------------#
chip.set_icasn_dacunits(0,0)
chip.set_vreset_dacunits(35,0)
chip.set_ireset_dacunits(2,1,0)
chip.set_ithr_dacunits(5,0)
chip.set_idb_dacunits(50,0)

chip.write_conf()

chip['CONF']['DEF_CONF_N'] = 1
chip['CONF'].write()

****************************************************POWER_STATUS*********************************************************

In [2]:
#chip['BiasSF'].set_current(100, unit='uA') # SET THE SOURCE FOLLOWER READOUT CURRENT
chip.get_power_status()

{'BiasSF [V]': 0.8971388147553919,
 'BiasSF [mA]': -0.32389637762237766,
 'VDDA [V]': 1.7921857260956175,
 'VDDA [mA]': 577.0899264452138,
 'VDDA_DAC [V]': 1.7915526135458166,
 'VDDA_DAC [mA]': 567.1670178167402,
 'VDDD [V]': 1.7921228324974925,
 'VDDD [mA]': 569.4866960087318,
 'VDDP [V]': 1.7913174844533601,
 'VDDP [mA]': 575.548350884382,
 'VPC [V]': 0.8968192194291438,
 'VPC [mA]': -0.32002818589178506,
 'VPCSWSF [V]': 0.8973983967065867,
 'VPCSWSF [mA]': -0.32130961215949544}

****************************ENABLE_COLUMS, SET MASKS, SELECT WHICH COLUMN AND ROW TO INJECT******************************

In [4]:
# SELECT WHICH DOUBLE COLUMNS TO ENABLE
chip['CONF_SR']['EN_PMOS_NOSF'].setall(False)
chip['CONF_SR']['EN_PMOS'].setall(False)
chip['CONF_SR']['EN_COMP'].setall(False)
chip['CONF_SR']['EN_HV'].setall(True)
chip['CONF_SR']['EN_OUT'].setall(False) #ENABLES OR DISABLES THE NORMAL OUTPUT PADS, ACTIVE LOW
chip['CONF_SR']['nEN_OUT'].setall(True) #ENABLES OR DISABLES THE COMPLEMENTARY OUTPUT PADS, ACTIVE LOW
chip['CONF_SR']['EN_HITOR_OUT'].setall(True) #ENABLES OR DISABLES THE NORMAL HITOR PADS, HITOR0-3 =  1-4 flavor, ACTIVE LOW
chip['CONF_SR']['nEN_HITOR_OUT'].setall(True) #ENABLES OR DISABLES THE COMPLEMENTARY HITOR PADS, ACTIVE LOW

#chip['CONF_SR']['EN_HV'][9] = 1
chip['CONF_SR']['EN_HV'].setall(True)
#chip['CONF_SR']['EN_HITOR_OUT'][1] = 0

# SELECT WHICH PHYSICAL COLUMNS, ROWS, DIAGONALS TO MASK
# TO MASK ONE PIXEL, MASKV, MASKH and MASKD OF THIS PIXEL SHOULD BE 0 (FALSE)
# THE MASKD NUMBER OF THE PIXEL WE WANT TO MASK (or UNMASK), IS GIVEN BY: MASKD = PHYSCOL- PHYSROW
# IF PHYSCOL-PHYSROW<0, then MASKD = 448+PHYSCOL-PHYSROW
chip['CONF_SR']['MASKD'].setall(True)
chip['CONF_SR']['MASKH'].setall(True)
chip['CONF_SR']['MASKV'].setall(True)

# TO USE THE MASK FUNCTION YOU MUST INPUT THE FLAVOR, COLUMN AND ROW
# THE FLAVOR NUMERS IS: 0 FOR PMOS_NOSF, 1 FOR PMOS, 2 FOR COMP, 3 FOR HV


# SELECT WHICH PHYSICAL COLUMS TO INJECT
# INJ_IN_MON_L AND INJ_IN_MON_L SELECT THE LEFT AND RIGHT SPECIAL ANALOG MONITORING PIXELS
chip['CONF_SR']['COL_PULSE_SEL'].setall(False)

chip['CONF_SR']['INJ_IN_MON_L'] = 1 # ENABLE INJECTION FOR THE ANALOG MONITORING PIXELS LEFT SIDE
chip['CONF_SR']['INJ_IN_MON_R'] = 1 # ENABLE INJECTION FOR THE ANALOG MONITORING PIXELS RIGHT SIDE

# SELECT WHICH PHYSICAL ROWS TO INJECT
# THE SPEXIAL PIXELS OUTA_MON3 to OUTA_MON0 CORRESPONT TO ROWS 223 to 220 FOR INJECTION
chip['CONF_SR']['INJ_ROW'].setall(False)
chip['CONF_SR']['INJ_ROW'][223:220] = True # FOR THE ANALOG MONITORING TOP PIXEL

# SELECT PHYSICAL COLUMNS AND ROWS FOR INJECTION WITH FUNCTION
chip.enable_injection(3,18,99)

# SELECT PHYSICAL COLUMN(S) FOR HITOR OUTPUT
# nMASKH (SO SETTING MASKH TO FALSE) ENABLES HITOR FOR THE SPECIFIC ROW
chip['CONF_SR']['DIG_MON_SEL'].setall(False)
#chip.enable_column_hitor(1,18)

chip.write_conf()

In [8]:
# chip['data_rx'].CONF_START_FREEZE = 64 #default 3
# chip['data_rx'].CONF_STOP_FREEZE = 100 #default 40
# chip['data_rx'].CONF_START_READ = 66 #default 6
# chip['data_rx'].CONF_STOP_READ = 68 #default 7
# chip['data_rx'].CONF_STOP = 105 #default 45

#chip['data_rx'].reset()
#chip['CONF']['AB_SELECT'] = 1
#chip['CONF'].write()

chip['data_rx'].set_en(True)

x = chip['fifo'].get_data()

ix = chip.interpret_data(x)
print(len(ix))
print(ix)

0
[]


************CONFIGURE THE FRONT END CURRENT AND VOLTAGE BIASES, CONFIGURE THE HEIGHT OF THE INJECTION PULSE**************

In [10]:
## SET THE INJECTION PULSE AMPLITUDE
## 128-bit DAC (7-bit binary equivalent)
## SET THE VOLTAGES IN ONE HOT ENCODING, ONLY ONE BIT ACTIVE AT A TIME.
## V = (127/1.8)*#BIT
# The default values are VL=44, VH=79, VH-VL=35
# VDAC LSB=14.17mV, Cinj=230aF, 1.43e-/mV, ~710e-
chip.set_vl_dacunits(40,1)
chip.set_vh_dacunits(80,1)

####### CONFIGURE THE FRONT END ######

# SET VRESET_P, THIS IS THE BASELINE OF THE FRONT END INPUT, ONE HOT ENCODING
chip.set_vreset_dacunits(35,1) #1V

## 128-bit DAC (7-bit binary equivalent)
## SET THE CURRENTS USING THERMOMETER ENCODING, I = #BITS_ACTIVE*140nA*SCALING, SCALING IS DIFFERENT FOR EACH CURRENT
## SCALING: IBIAS=10, IDB=16, ITHR=0.125, ICASN=4, IRESET=0.03125
## ACTIVE BITS SHOULD BE SET STARTING FROM THE MIDDLE e.g. for 15 active bits, (128-15)/2=56,5 so 56zeros,15ones,57zeros
## Thus, Ix[71:57] = True

# SET ICASN, THIS CURRENT CONTROLS THE OUTPUT BASELINE, BE CAREFUL NOT TO SET IT TO HIGH
# ALWAYS MONITOR THE POWER AFTER SETTING ICASN. IF VDDD IS SEVERAL mA, REDUCE IT UNTIL IT RETURNS TO 0
# ICASN MAINLY CONTROLS THE THRESHOLD
chip.set_icasn_dacunits(0,1) #4.375nA # approx 1.084V at -3V backbias, 600mV at 0V backbias

# SET IRESET, THIS CURRENT CONTROLS THE RESET RATE OF THE FRONT END INPUT (ALSO THE THRESHOLD)
chip.set_ireset_dacunits(2,1,1) #270pA, HIGH LEAKAGE MODE, NORMAL SCALING, 0 = LOW LEAKAGE MODE, SCALING*0.01

# SET ITHR, THIS CURRENT CONTROLS THE RESET RATE OF THE OUTPUT (AND THE THRESHOLD)
chip.set_ithr_dacunits(5,1) #680pA

# SET ITHR, THIS CURRENT CONTROLS THE BIASING OF THE DISCRIMINATOR (AND THE THRESHOLD)
chip.set_idb_dacunits(20,1) #500nA

# SET IBIAS, THIS CURRENT IS THE DC CURRENT OF THE MAIN BRANCH OF THE FRONT END THAT PROVIDES AMPLIFICATION
# IT CONTROLS MAINLY THE RISE TIME
chip.set_ibias_dacunits(50,1) #500nA

############ ENABLE THE DAC CURRENT MONITORING ###########
# chip['CONF_SR']['SWCNTL_DACMONI'] = 0

########## SET THE BIAS CURRENTS OF THE TWO STAGE SOURCE FOLLOWER THAT BUFFERS THE ANALOG MONITORING VOLTAGES #########
# CONTROLS THE RESPONSE TIME AND THE LEVEL SHIFT OF THE BUFFER
# chip['CONF_SR']['SET_IBUFN_L'] = 0b1001
# chip['CONF_SR']['SET_IBUFP_L'] = 0b0101

chip.write_conf()

2019-04-03 22:17:23,850 [INFO ] (MainThread) vl = 0.951929133858V
2019-04-03 22:17:23,859 [INFO ] (MainThread) vh = 1.51885826772V
2019-04-03 22:17:23,861 [INFO ] (MainThread) vreset = 1.05106299213V
2019-04-03 22:17:23,865 [INFO ] (MainThread) icasn = 0
2019-04-03 22:17:23,866 [INFO ] (MainThread) icasn = 4.375nA
2019-04-03 22:17:23,872 [INFO ] (MainThread) ireset = 2 high leakage mode
2019-04-03 22:17:23,873 [INFO ] (MainThread) ireset = 0.1025390625nA, high leakage mode
2019-04-03 22:17:23,877 [INFO ] (MainThread) ithr = 5
2019-04-03 22:17:23,881 [INFO ] (MainThread) ithr = 0.8203125nA
2019-04-03 22:17:23,883 [INFO ] (MainThread) idb = 20
2019-04-03 22:17:23,885 [INFO ] (MainThread) idb = 367.5nA
2019-04-03 22:17:23,889 [INFO ] (MainThread) ibias = 50
2019-04-03 22:17:23,890 [INFO ] (MainThread) ibias = 557.8125nA


******************INITIALIZE COMMUNICATION, RECEIVE DATA (CHECK IF DATA ARE SENT WITH EACH EXCECUTION)*******************

In [11]:
for _ in range(10):
    chip['fifo'].reset()
    time.sleep(0.002)
time.sleep(1)
x = chip['fifo'].get_data()
ix = chip.interpret_data(x)
print(len(ix))
uni,cnt=np.unique(ix[["col","row"]],return_counts=True)
arg=np.argsort(cnt)
for a_i,a in enumerate(arg[::-1]):
    print uni[a],cnt[a]
    chip.mask(3,uni[a][0],uni[a][1])
chip.write_conf()   

155490
(59, 212) 122783
(28, 159) 26835
(81, 180) 3415
(35, 126) 1659
(55, 210) 228
(111, 54) 220
(107, 7) 179
(38, 162) 59
(67, 119) 48
(104, 77) 26
(32, 95) 13
(58, 2) 9
(23, 215) 3
(64, 100) 3
(74, 164) 3
(47, 204) 2
(97, 188) 2
(56, 59) 1
(123, 244) 1


AssertionError: Column must be between 0 and 111

In [12]:
#### option (external ICASN)
chip['DACMON_ICASN'].set_current(0, unit='uA')
chip['CONF_SR']['SWCNTL_ICASN']=True
chip.write_conf()
time.sleep(1)
print chip['DACMON_ICASN'].get_voltage()
print chip['DACMON_ICASN'].get_current()

0.89684425296
-0.000323668386014


****************************SET THE INJECTION PULSE PARAMETERS, INJECT x times and get the data**************************

In [13]:
#chip['CONF_SR']['COL_PULSE_SEL'].setall(False)
#chip['CONF_SR']['INJ_ROW'].setall(False)
#chip['CONF_SR']['INJ_ROW'][223:220]=True
#chip.write_conf()
delay=800
width=250
repeat=100
chip['inj'].set_delay(delay)
chip['inj'].set_width(width)
chip['inj'].set_repeat(repeat)
chip['inj'].set_phase(0)
chip['inj'].set_en(0)

col=18
row=25
chip['CONF_SR']['INJ_ROW'].setall(False)
chip['CONF_SR']['COL_PULSE_SEL'].setall(False)
chip.enable_injection(3,col,row)

chip.set_vl_dacunits(35,1)
chip.set_vh_dacunits(100,1)
chip.write_conf()
for _ in range(5):
    chip['fifo'].reset()
    time.sleep(0.002)
chip["inj"].start()
while not chip['inj'].is_ready:
    time.sleep(0.001)
time.sleep(0.2)
print chip['fifo'].get_fifo_size()
x = chip['fifo'].get_data()
ix = chip.interpret_data(x)
uni,cnt=np.unique(ix[["col","row"]],return_counts=True)
arg=np.argsort(cnt)
for a_i,a in enumerate(arg[::-1]):
    print uni[a],cnt[a]

2019-04-03 22:17:51,245 [INFO ] (MainThread) vl = 0.881062992126V
2019-04-03 22:17:51,257 [INFO ] (MainThread) vh = 1.80232283465V
2019-04-03 22:17:51,600 [WARNI] (MainThread) Deprecated: Use get_FIFO_SIZE()


2637824
(96, 210) 536
(94, 144) 535
(77, 111) 535
(77, 66) 535
(91, 101) 535
(92, 129) 535
(91, 82) 535
(93, 161) 535
(94, 106) 535
(78, 96) 535
(78, 201) 534
(78, 54) 534
(90, 80) 534
(90, 156) 534
(78, 70) 534
(76, 160) 534
(93, 176) 534
(79, 70) 533
(75, 52) 533
(89, 64) 533
(88, 156) 533
(88, 77) 533
(88, 42) 533
(81, 179) 533
(82, 44) 533
(91, 163) 533
(86, 1) 533
(87, 12) 533
(63, 194) 532
(61, 23) 532
(87, 38) 532
(63, 202) 532
(76, 218) 532
(87, 222) 532
(99, 100) 531
(98, 69) 531
(71, 111) 531
(70, 21) 531
(76, 44) 531
(99, 190) 531
(81, 146) 530
(23, 70) 530
(82, 169) 530
(61, 75) 530
(82, 182) 530
(25, 157) 530
(24, 164) 530
(72, 182) 530
(23, 192) 530
(23, 154) 530
(23, 134) 530
(72, 168) 530
(7, 60) 530
(69, 74) 530
(14, 26) 530
(11, 77) 530
(10, 202) 530
(8, 119) 530
(22, 185) 530
(72, 69) 530
(22, 149) 530
(22, 62) 530
(24, 183) 529
(63, 84) 529
(16, 74) 529
(82, 199) 529
(13, 185) 529
(99, 59) 529
(55, 169) 529
(24, 146) 529
(7, 166) 529
(16, 45) 529
(103, 213) 529
(6, 

(46, 23) 2
(33, 213) 2
(85, 144) 2
(35, 198) 2
(99, 154) 2
(39, 25) 2
(98, 147) 2
(32, 44) 2
(98, 159) 2
(32, 160) 2
(20, 87) 2
(49, 216) 2
(50, 17) 2
(92, 135) 2
(40, 152) 2
(28, 116) 2
(5, 156) 1
(102, 122) 1
(18, 154) 1
(110, 181) 1
(67, 59) 1
(40, 53) 1
(66, 23) 1
(40, 114) 1
(65, 17) 1
(5, 172) 1
(99, 192) 1
(110, 220) 1
(34, 40) 1
(32, 161) 1
(99, 18) 1
(111, 130) 1
(70, 115) 1
(111, 103) 1
(97, 96) 1
(96, 74) 1
(100, 66) 1
(34, 153) 1
(3, 108) 1
(40, 205) 1
(3, 181) 1
(36, 10) 1
(29, 34) 1
(27, 177) 1
(77, 147) 1
(110, 160) 1
(16, 116) 1
(106, 184) 1
(86, 64) 1
(86, 33) 1
(22, 59) 1
(55, 19) 1
(85, 154) 1
(21, 100) 1
(49, 34) 1
(23, 102) 1
(16, 134) 1
(49, 98) 1
(54, 88) 1
(49, 161) 1
(52, 69) 1
(53, 160) 1
(19, 38) 1
(106, 152) 1
(80, 179) 1
(27, 84) 1
(79, 131) 1
(93, 105) 1
(77, 127) 1
(41, 195) 1
(91, 200) 1
(18, 156) 1
(26, 60) 1
(59, 167) 1
(24, 219) 1
(12, 198) 1
(58, 221) 1
(24, 218) 1
(58, 121) 1
(80, 43) 1
(104, 171) 1
(55, 223) 1
(88, 120) 1
(20, 121) 1


In [ ]:
for _ in range(5):
    chip['fifo'].reset()
    time.sleep(0.002)
chip.reset_ibias()

chip.auto_mask()

In [ ]:
delay=800
width=250
repeat=100
chip['inj'].set_delay(delay)
chip['inj'].set_width(width)
chip['inj'].set_repeat(repeat)
chip['inj'].set_phase(0)
chip['inj'].set_en(0)

col=18
row=25
chip.set_vl_dacunits(40,1)
chip.set_vh_dacunits(54,1)
chip.set_idb_dacunits(20,1)

chip['CONF_SR']['MASKV'].setall(False)
chip['CONF_SR']['MASKH'].setall(False)
chip['CONF_SR']['MASKD'].setall(False)
chip['CONF_SR']['MASKH'][row]=True
chip['CONF_SR']['EN_HV'][col//2]=True
chip.write_conf()
for _ in range(5):
    chip['fifo'].reset()
    time.sleep(0.002)
chip.reset_ibias()

import tjmonopix.scans.injection_scan
reload(tjmonopix.scans.injection_scan)
scans=tjmonopix.scans.injection_scan.InjectionScan(dut=chip)

pix=[3,col,row]
injlist=np.arange(73,0,-1)
thlist= None #np.arange(80,20,-1)
phaselist=None
scans.start(pix=pix,injlist=injlist,thlist=thlist,phaselist=phaselist,with_mon=False,n_mask_pix=1,debug=3)

In [ ]:
chip.enable_injection(3,col,row)
chip.set_idb_dacunits(20,1)
chip.write_conf()
for _ in range(5):
    chip['fifo'].reset()
    time.sleep(0.002)
injlist=np.arange(73,0,-1)
inj_low=chip.get_vl_dacunits()
cnt=[]
for inj in injlist:
    chip.set_vh_dacunits(inj+inj_low,0)
    chip.write_conf()
    for _ in range(5):
        chip['fifo'].reset()
        time.sleep(0.002)
    chip.set_monoread()
    chip["inj"].start()
    while not chip['inj'].is_ready:
        time.sleep(0.001)
    time.sleep(0.2)
    x = chip['fifo'].get_data()
    ix = chip.interpret_data(x)
    ix_inj=ix[np.bitwise_and(ix["col"]==col,ix["row"]==row)]
    print inj,len(ix),len(ix_inj),np.average((ix_inj["te"]-ix_inj["le"]) & 0x3F)
    cnt.append(len(ix_inj))
    #uni,cnt=np.unique(,col","row"]],return_counts=True)
    #    print uni[a],cnt[a]

In [ ]:
import tjmonopix.analysis.interpreter_idx
reload(tjmonopix.analysis.interpreter_idx)
import tjmonopix.analysis.event_builder_inj
reload(tjmonopix.analysis.event_builder_inj)
fraw=scans.output_filename+".h5"
fhit=scans.output_filename[:-4]+"hit.h5"
fev=scans.output_filename[:-4]+"ev.h5"
tjmonopix.analysis.interpreter_idx.interpret_idx_h5(fraw,fhit)
tjmonopix.analysis.event_builder_inj.build_inj_h5(fhit,fraw=fraw,fout=fev)
import tables as tb
with tb.open_file(fev) as f:
    hits=f.root.Hits[:]
print len(hits)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
uni,u_cnt=np.unique(hits["inj"],return_counts=True)
data=np.zeros(len(injlist))
for u_i,u in enumerate(uni):
    a=np.argmin(np.abs(injlist-u))
    data[a]=u_cnt[u_i]
plt.plot(injlist,data,"o")
plt.plot(injlist,cnt,"o")

******************************************INJECTION SCANS -  S-CURVE*****************************************************

In [ ]:
%matplotlib inline
import numpy as np

flavor=3
col=18
startrow=180
rownumber=1
VL=40
VHLrange=40
start_dif=0
repeat=500
delay=500
width=250
noise_en=0
analog_en=1
sleeptime=repeat*0.00005
DUtoe=20.18
#DUtoe=23

xhits = range(start_dif,VHLrange+start_dif+1)

hits =chip.inj_scan_1pix(flavor, col, startrow, VL, VHLrange, start_dif, delay, width, repeat, noise_en, analog_en, sleeptime)
print hits
#plot.plot_scurve(col, startrow, xhits, hits[0], repeat, VHLrange/2+start_dif, 0.5, 10.0, DUtoe,False)

In [ ]:
arg=np.argwhere(np.array(list(chip['CONF_SR']['SET_IDB'].to01()),dtype=int))
low=arg[0,0]
high=arg[-1,0]
dacunits_low=128-low*2
dacunits_high= (high -(128 // 2))*2
if dacunits_low!=dacunits_high:
    logger.info("WARN unusual dac setting")
return dacunits_low